# IMDB Movie Review Dataset

This is a library to download and parse the [IMDB's Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) dataset and a demo of a transformer based model. The dataset has 25K training, and 25K test dataset, plus 50K unlabeled examples.

It's inspired on [Keras' Text classification with Transformer](https://keras.io/examples/nlp/text_classification_with_transformer/) demo.

## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

Notice the directory `${HOME}/Projects/gomlx` is where the GoMLX code is copied by default in [its Docker](https://hub.docker.com/repository/docker/janpfeifer/gomlx_jupyterlab/general).

In [1]:
!*rm -f go.work && go work init && go work use . "${HOME}/Projects/gomlx" "${HOME}/Projects/gopjrt"
%goworkfix

	- Added replace rule for module "github.com/gomlx/gomlx" to local directory "/home/janpf/Projects/gomlx".
	- Added replace rule for module "github.com/gomlx/gopjrt" to local directory "/home/janpf/Projects/gopjrt".


## Data Preparation

### Downloading data files

To download, uncompress and untar to the local directory, simply do the following. Notice if it's already downloaded in the given `--data` directory, it returns immediately.

In [2]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
    "github.com/gomlx/gomlx/pkg/support/fsutil"
    "github.com/janpfeifer/must"

    _ "github.com/gomlx/gomlx/backends/default"
)

var (
	flagDataDir    = flag.String("data", "~/tmp/imdb", "Directory to cache downloaded and generated dataset files.")
	flagEval       = flag.Bool("eval", true, "Whether to evaluate the model on the validation data in the end.")
	flagVerbosity  = flag.Int("verbosity", 1, "Level of verbosity, the higher the more verbose.")
	flagCheckpoint = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
)

func AssertDownloaded() {
    *flagDataDir = must.M1(fsutil.ReplaceTildeInDir(*flagDataDir))
    if !fsutil.MustFileExists(*flagDataDir) {
        must.M(os.MkdirAll(*flagDataDir, 0777))
    }
    must.M(imdb.Download(*flagDataDir))
}

%%
AssertDownloaded()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.


### Sampling some examples

It creates a small dataset and print out some random examples.

It also defines the `DType`, used for all internal representations of the model, and the flag `--max_len` that defines the maximum number of tokens used per observation. This will beused in the modeling later.

In [3]:
import "github.com/gomlx/gomlx/examples/imdb"

%%
AssertDownloaded()
imdb.PrintSample(3)

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
┌────────────────────────────────────────────────────────────┐
│                                                            │
│    [Sample 0 - label 0]                                    │
│    poorly filmed just so you don t think i can t find      │
│    anything good here on the plus side anne jeffreys is    │
│    very sexy in her all too brief parts of this film       │
│    not sure if it is actually her singing or someone       │
│    else but whoever it was had a very pretty voice ms      │
│    jeffreys also had a couple of nice acting moments       │
│    the script needed either a lot more of her or to        │
│    remove her character altogether as it was her nice      │
│    few moments weren t enough to help the film lastly      │
│    there is mr scott he looks fantastic in this film       │
│    and is the no nonsense

## Training

We will create 3 different types of models for this demo: **Bag of Words** (**"bow"**), **Convolutionals** (**"cnn"**) and **Transformers** (**"transformer"**).

### Model Configuration

As with other demos we leverage the `context.Context` object to store all model and training parameters. 
One can set specific parameters using the `-set` command line flag.

The [`imdb.CreateDefaultContext()`](https://github.com/gomlx/gomlx/blob/main/examples/imdb/train.go) method sets all the default values for the hyperparameters that may be used by any of the 3 model types. The parameter "model" specify the model type.

In [4]:
import (
    "golang.org/x/exp/maps"
    "github.com/gomlx/gomlx/pkg/ml/context"
)

// settings is bound to a "-set" flag to be used to set context hyperparameters.
var settings = commandline.CreateContextSettingsFlag(imdb.CreateDefaultContext(), "set")

// ContextFromSettings is the default context (createDefaultContext) changed by -set flag.
// It also returns the list of parameters changed by -set in paramsSet: we use this later to avoid loading over the values from checkpoints.
func ContextFromSettings() (ctx *context.Context, paramsSet []string) {
    ctx = imdb.CreateDefaultContext()
    paramsSet = must.M1(commandline.ParseContextSettings(ctx, *settings))
    return ctx, paramsSet
}

%% -set="model=cnn"
fmt.Printf("Model types: %q\n", maps.Keys(imdb.ValidModels))
ctx, _ := ContextFromSettings()
fmt.Println(commandline.SprintContextSettings(ctx))

Model types: ["bow" "cnn" "transformer"]
	"/activation": (string) 
	"/adam_dtype": (string) 
	"/adam_epsilon": (float64) 1e-07
	"/batch_size": (int) 32
	"/cnn_dropout_rate": (float64) 0.5
	"/cnn_normalization": (string) 
	"/cnn_num_layers": (float64) 5
	"/cosine_schedule_steps": (int) 0
	"/dropout_rate": (float64) 0.1
	"/eval_batch_size": (int) 200
	"/fnn_dropout_rate": (float64) 0.3
	"/fnn_normalization": (string) 
	"/fnn_num_hidden_layers": (int) 2
	"/fnn_num_hidden_nodes": (int) 32
	"/fnn_residual": (bool) true
	"/imdb_content_max_len": (int) 200
	"/imdb_include_separators": (bool) false
	"/imdb_mask_word_task_weight": (float64) 0
	"/imdb_max_vocab": (int) 20000
	"/imdb_token_embedding_size": (int) 32
	"/imdb_use_unsupervised": (bool) false
	"/imdb_word_dropout_rate": (float64) 0
	"/l1_regularization": (float64) 0
	"/l2_regularization": (float64) 0
	"/learning_rate": (float64) 0.0001
	"/model": (string) cnn
	"/normalization": (string) layer
	"/num_checkpoints": (int) 3
	"/optimizer"

### Bag Of Words Model (bow)

This is the simplest model we are going to train: it embeds each token of the sentence (default size of the is 32 numbers) sum them up, and pass that through a FNN.

The [code in `imdb.BagOfWordsModelGraph`](https://github.com/gomlx/gomlx/blob/main/examples/imdb/model_bagofwords.go) looks like this:

```go
// BagOfWordsModelGraph builds the computation graph for the "bag of words" model: simply the sum of the embeddings
// for each token included.
func BagOfWordsModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
	embed, _ := EmbedTokensGraph(ctx, inputs[0])

	// Take the max over the content length, and put an FNN on top.
	// Shape transformation: [batch_size, content_len, embed_size] -> [batch_size, embed_size]
	embed = ReduceMax(embed, 1)
	logits := fnn.New(ctx, embed, 1).Done()
	return []*Node{logits}
}
```

We played a bit with the hyperparameters to get to ~85% accuracy on the validation data.

The [code for `imdb.TrainModel` is here](https://github.com/gomlx/gomlx/blob/main/examples/imdb/train.go).
It's a straight forward GoMLX training loop.

In [5]:
%% --set="model=bow;l2_regularization=1e-3;learning_rate=1e-4;normalization=none;train_steps=10000"
ctx, paramsSet := ContextFromSettings()
imdb.TrainModel(ctx, *flagDataDir, *flagCheckpoint, paramsSet, *flagEval, *flagVerbosity)

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Backend "stablehlo":	stablehlo:cuda - PJRT "cuda" plugin (/home/janpf/.local/lib/gomlx/pjrt/pjrt_c_api_cuda_plugin.so) v0.76 [StableHLO]
Model: bow


         7% [=>......................................] (189 steps/s) [9s:48s] [step=719] [loss+=0.801] [~loss+=0.787] [~loss=0.676] [~acc=59.05%]         

       100% [========================================] (130 steps/s) [step=9999] [loss+=0.289] [~loss+=0.328] [~loss=0.277] [~acc=88.51%]        1%]        


Metric: accuracy

Metric: loss

	[Step 10000] median train step: 4563 microseconds

Results on train-eval:
	Mean Loss+Regularization (#loss+): 0.246
	Mean Loss (#loss): 0.195
	Mean Accuracy (#acc): 93.48%
Results on test-eval:
	Mean Loss+Regularization (#loss+): 0.381
	Mean Loss (#loss): 0.33
	Mean Accuracy (#acc): 85.72%


### Convolution Model (cnn)

The function [`imdb.CnnModelGraph`](https://github.com/gomlx/gomlx/blob/main/examples/imdb/model_cnn.go) creates a 1D convolution model, with arbitrary number of convolutions. After the convolution, it behaves the same way as the Bag Of Words model.

The core of the convolution model looks like this:

```go
	// 1D Convolution: embed is [batch_size, content_len, embed_size].
	numConvolutions := context.GetParamOr(ctx, "cnn_num_layers", 5)
	logits := embed
	for convIdx := range numConvolutions {
		ctx := ctx.Inf("%03d_conv", convIdx)
		residual := logits
		if convIdx > 0 {
			logits = NormalizeSequence(ctx, logits)
		}
		logits = layers.Convolution(ctx, embed).KernelSize(7).Filters(embedSize).Strides(1).Done()
		logits = activations.ApplyFromContext(ctx, logits)
		if dropoutNode != nil {
			logits = layers.Dropout(ctx, logits, dropoutNode)
		}
		if residual.Shape().Equal(logits.Shape()) {
			logits = Add(logits, residual)
		}
	}

	// Take the max over the content length, and put an FNN on top.
	// Shape transformation: [batch_size, content_len, embed_size] -> [batch_size, embed_size]
	logits = ReduceMax(logits, 1)
	logits = fnn.New(ctx, logits, 1).Done()
	logits.AssertDims(batchSize, 1)
```

Notice how well it can overfit to the training data ... but it doesn't help the test results. To improve this one needs some careful regularization.

In [6]:
%% --set="model=cnn;l2_regularization=1e-3;learning_rate=1e-4;normalization=layer;train_steps=10000"
ctx, paramsSet := ContextFromSettings()
imdb.TrainModel(ctx, *flagDataDir, *flagCheckpoint, paramsSet, *flagEval, *flagVerbosity)

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Backend "stablehlo":	stablehlo:cuda - PJRT "cuda" plugin (/home/janpf/.local/lib/gomlx/pjrt/pjrt_c_api_cuda_plugin.so) v0.76 [StableHLO]
Model: cnn


         7% [=>......................................] (165 steps/s) [14s:56s] [step=719] [loss+=1.51] [~loss+=1.46] [~loss=0.781] [~acc=51.26%]          

       100% [========================================] (95 steps/s) [step=9999] [loss+=0.278] [~loss+=0.306] [~loss=0.105] [~acc=96.54%]        47%]         


Metric: accuracy

Metric: loss

	[Step 10000] median train step: 4881 microseconds

Results on train-eval:
	Mean Loss+Regularization (#loss+): 0.239
	Mean Loss (#loss): 0.0398
	Mean Accuracy (#acc): 98.87%
Results on test-eval:
	Mean Loss+Regularization (#loss+): 0.942
	Mean Loss (#loss): 0.743
	Mean Accuracy (#acc): 83.70%


### Transformer Model

Finally a Transformer version of the model, as defined in the ["Attention Is All You Need"](https://arxiv.org/abs/1706.03762) famous paper. 

Notice it's not significantly better than our previous simple Bag-Of-Words model. Likely because there is not enough data for the transformer to make any difference. The success of transformers in large-language-models is in large part due to the training with huge amounts of unsupervised (or self-supervised) data, but that is beyond the scope of this small test.

The code is in [`imdb.TransformerModelGraph`](https://github.com/gomlx/gomlx/blob/main/examples/imdb/model_transformer.go), and the core of it looks like this:

```go
    ...
	// Add the requested number of attention layers.
	numAttLayers := context.GetParamOr(ctx, "transformer_num_att_layers", 1)
	numAttHeads := context.GetParamOr(ctx, "transformer_num_att_heads", 2)
	attKeySize := context.GetParamOr(ctx, "transformer_att_key_size", 8)
	for layerNum := range numAttLayers {
		// Each layer in its own scope.
		ctx := ctx.Inf("%03d_attention_layer", layerNum)
		residual := embed
		embed = layers.MultiHeadAttention(ctx.In("000_attention"), embed, embed, embed, numAttHeads, attKeySize).
			SetKeyMask(mask).SetQueryMask(mask).
			SetOutputDim(embedSize).
			SetValueHeadDim(embedSize).Done()
		if dropoutNode != nil {
			embed = layers.Dropout(ctx.In("001_dropout"), embed, dropoutNode)
		}
		embed = NormalizeSequence(ctx.In("002_normalization"), embed)
		attentionOutput := embed

		// Transformers recipe: 2 dense layers after attention.
		embed = fnn.New(ctx.In("003_fnn"), embed, embedSize).NumHiddenLayers(1, embedSize).Done()
		if dropoutNode != nil {
			embed = layers.Dropout(ctx.In("004_dropout"), embed, dropoutNode)
		}
		embed = Add(embed, attentionOutput)
		embed = NormalizeSequence(ctx.In("005_normalization"), embed)

		// Residual connection:
		if layerNum > 0 {
			embed = Add(residual, embed)
		}
	}
    ...
```

With only 5000 steps we got ~87% on the test data -- and significant overfitting as well.

In [7]:
%% --set="model=transformer;normalization=none;activation=swish;l2_regularization=1e-3;cnn_dropout_rate=0.5;fnn_dropout_rate=0.3;learning_rate=1e-4;train_steps=5000"
ctx, paramsSet := ContextFromSettings()
imdb.TrainModel(ctx, *flagDataDir, *flagCheckpoint, paramsSet, *flagEval, *flagVerbosity)

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Backend "stablehlo":	stablehlo:cuda - PJRT "cuda" plugin (/home/janpf/.local/lib/gomlx/pjrt/pjrt_c_api_cuda_plugin.so) v0.76 [StableHLO]
Model: transformer


        14% [====>...................................] (161 steps/s) [13s:26s] [step=724] [loss+=1.05] [~loss+=1.21] [~loss=0.816] [~acc=51.65%]        

       100% [========================================] (82 steps/s) [step=4999] [loss+=0.599] [~loss+=0.565] [~loss=0.227] [~acc=91.92%]        9%]        


Metric: accuracy

Metric: loss

	[Step 5000] median train step: 5063 microseconds

Results on train-eval:
	Mean Loss+Regularization (#loss+): 0.635
	Mean Loss (#loss): 0.299
	Mean Accuracy (#acc): 88.12%
Results on test-eval:
	Mean Loss+Regularization (#loss+): 0.717
	Mean Loss (#loss): 0.381
	Mean Accuracy (#acc): 82.70%
